<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/SVM-SOFTMARGIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import time
t1 = time.time()
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

try:
  COLAB = True
  import torch
  from torch import nn, optim
  from torch.utils.data import DataLoader, Dataset
  import matplotlib.pyplot as plt
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.datasets import load_breast_cancer
  print(f">>>> You are on CoLaB with torch version {torch.__version__}")
except Exception as e:
  COLAB = False
  print(f">>>> {type(e)}: {e}\n>>>> Please corect {type(e)} and reload your device")

def timefmt(t: float = 231.781)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}, min: {m:>02}, sec: {s:>05.2f}"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 256 if device == torch.device("cuda") else 32
print(f">>>> Available device: {device}")
!nvidia-smi
t2 = time.time()
print(f">>>> time elapsed: {timefmt(t2 - t1)}")

Mounted at /content/drive
>>>> You are on CoLaB with torch version 1.13.0+cu116
>>>> Available device: cpu
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

>>>> time elapsed: hrs: 0, min: 00, sec: 03.00


#### In this notebook we present the Implementantion of the support vector machine with a sofmargin loss function from scratch in Pytorch. We train on the breast cancer dataset (Available in scikit learn repo) 

In [ ]:
cancer = load_breast_cancer()
cancer_data = pd.DataFrame(data = cancer.data, columns = cancer.feature_names)
cancer_data["target"] = cancer["target"]
#display(cancer_data.head(10))

In [11]:
print(f">>>> {cancer.target_names}\t {cancer_data.target.unique()}")

>>>> ['malignant' 'benign']	 [0 1]


In [13]:
# rescale the target into [1, -1] for SVM classifier
cancer_data["target"] = 2 * cancer_data["target"] - 1

In [ ]:
print(cancer_data.target[:30])

In [18]:
len(list(cancer_data.columns)) - 1

30

# Pytorch soft-margin SVM clasifier

In [20]:
class SVM_soft(nn.Module):
  def __init__(self, in_dim):
    super().__init__()
    self.svm_soft = nn.Linear(in_dim, 1)
  
  def forward(self, x: torch.Tensor)->torch.Tensor:
    return self.svm_soft(x)

in_dim = 30
x = torch.randn(size = (BATCH_SIZE, in_dim))
model = SVM_soft(in_dim)
assert model(x).shape == (BATCH_SIZE, 1)

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(cancer_data.drop(columns = ["target"]), cancer_data["target"], test_size = 0.25)

In [27]:
X_train, X_test, Y_train, Y_test = torch.from_numpy(X_train.values).float(),torch.from_numpy(X_test.values).float(),\
torch.from_numpy(Y_train.values).float(), torch.from_numpy(Y_test.values).float()
print(f">>>> X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}\
\n>>>> X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

>>>> X_train shape: torch.Size([426, 30]), Y_train shape: torch.Size([426])
>>>> X_test shape: torch.Size([143, 30]), Y_test shape: torch.Size([143])


#Customized Pytorch DataLoader

In [28]:
class MyDataset(Dataset):
  def __init__(self, X, Y):
    super().__init__()
    self.X = X
    self.Y = Y
  
  def __len__(self):
    return len(self.Y)
  
  def __getitem__(self, index):
    X = self.X[index]
    Y = self.Y[index]
    return X, Y

train_data, test_data = MyDataset(X_train, Y_train), MyDataset(X_test, Y_test)

train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = False)

x_train_batch, y_train_batch = next(iter(train_loader))
x_test_batch, y_test_batch = next(iter(test_loader))
print(f">>>> x_train_batch: {x_train_batch.shape}, y_train_batch: {y_train_batch.shape}\
\n>>>> x_test_batch: {x_test_batch.shape}, y_test_batch: {y_test_batch.shape}")

>>>> x_train_batch: torch.Size([32, 30]), y_train_batch: torch.Size([32])
>>>> x_test_batch: torch.Size([32, 30]), y_test_batch: torch.Size([32])


# Pytorch Training loop

In [30]:
learning_rate = 1e-3
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)
C = 1

In [34]:
def Myaccuracy(y_true: torch.Tensor, y_preds: torch.Tensor):
  y_preds = 2 * (y_preds > 0) - 1
  acc = torch.eq(y_true, y_preds).sum().item()
  return (acc / len(y_true) * 100) 

In [40]:
def train_test_loop(model, train_loader, test_loader, optimizer, EPOCHS):
  model.train()
  total_train_loss = 0
  total_val_loss = 0
  total_samples = 0
  for epoch in range(EPOCHS):
    for batch, (data, labels) in enumerate(train_loader):
      data, labels = data.to(device), labels.to(torch.int64).to(device)
      logits = model(data)
      logits = logits.view(-1)
      train_loss = 0.5 * torch.norm(model.svm_soft.weight.squeeze())**2 + C * torch.clamp((1 - labels * logits), 0).mean()
      optimizer.zero_grad()
      train_loss.backward()
      total_train_loss+=train_loss
      optimizer.step()
      total_samples+=batch
    
    if epoch % 100 == 0:
      print(f">>>> End of train for epoch: {epoch}, train_loss:{total_train_loss/total_samples:.4f},\
      train accuracy: {Myaccuracy(labels, logits):.2f}")
    
    model.eval()
    with torch.inference_mode():
      for batch, (data, labels) in enumerate(test_loader):
        data, labels = data.to(device), labels.to(torch.int64).to(device)

        logits = model(data)
        logits = logits.view(-1)
        val_loss = 0.5 * torch.norm(model.svm_soft.weight.squeeze())**2 + C * torch.clamp((1 - labels * logits), 0).mean()
        total_val_loss+=val_loss

    if epoch % 100 == 0:
      print(f">>>> End of evaluation for epoch: {epoch}, validation_loss:{total_val_loss/total_samples:.4f},\
      validation accuracy: {Myaccuracy(labels, logits):.2f}")





# Training the SVM-soft margin for 1000 epochs

In [41]:
train_test_loop(model, train_loader, test_loader, optimizer, 1000)

>>>> End of train for epoch: 0, train_loss:0.0254,      train accuracy: 100.00
>>>> End of evaluation for epoch: 0, validation_loss:0.0142,      validation accuracy: 93.33
>>>> End of train for epoch: 100, train_loss:0.0293,      train accuracy: 100.00
>>>> End of evaluation for epoch: 100, validation_loss:0.0132,      validation accuracy: 100.00
>>>> End of train for epoch: 200, train_loss:0.0291,      train accuracy: 90.00
>>>> End of evaluation for epoch: 200, validation_loss:0.0126,      validation accuracy: 100.00
>>>> End of train for epoch: 300, train_loss:0.0294,      train accuracy: 90.00
>>>> End of evaluation for epoch: 300, validation_loss:0.0125,      validation accuracy: 100.00
>>>> End of train for epoch: 400, train_loss:0.0293,      train accuracy: 90.00
>>>> End of evaluation for epoch: 400, validation_loss:0.0124,      validation accuracy: 100.00
>>>> End of train for epoch: 500, train_loss:0.0292,      train accuracy: 90.00
>>>> End of evaluation for epoch: 500, vali

In [ ]:
# 